[1] https://www.freepublicapis.com/air-quality-api

In [2]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas
!pip install pandas numpy matplotlib scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 685.4/685.4 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.1/394.1 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.1 MB/s eta 0:00:0000:01
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 25.12.19
    Uninstalling flatbuffers-25.12.19:
      Successfully uninstalled flatbuffers-25.12.19
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 7.1 MB/s eta 0:00:00


In [3]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

import datetime as dt

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": ["temperature_2m", "rain", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", 
            "precipitation_probability", "cloud_cover", "cloud_cover_high", "cloud_cover_mid", "cloud_cover_low", "temperature_180m", 
            "temperature_80m", "temperature_120m", "soil_temperature_0cm", "soil_temperature_6cm", "soil_temperature_18cm", 
            "soil_moisture_0_to_1cm", "soil_temperature_54cm", "soil_moisture_1_to_3cm", "soil_moisture_3_to_9cm", 
            "soil_moisture_27_to_81cm", "soil_moisture_9_to_27cm"],
	"past_days": 92,
	"forecast_days": 16,
	"ref": "freepublicapis.com",
}
responses = openmeteo.weather_api(url, params=params)


In [4]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_rain = hourly.Variables(1).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(3).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(4).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(5).ValuesAsNumpy()
hourly_precipitation_probability = hourly.Variables(6).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(7).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(8).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(9).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(10).ValuesAsNumpy()
hourly_temperature_180m = hourly.Variables(11).ValuesAsNumpy()
hourly_temperature_80m = hourly.Variables(12).ValuesAsNumpy()
hourly_temperature_120m = hourly.Variables(13).ValuesAsNumpy()
hourly_soil_temperature_0cm = hourly.Variables(14).ValuesAsNumpy()
hourly_soil_temperature_6cm = hourly.Variables(15).ValuesAsNumpy()
hourly_soil_temperature_18cm = hourly.Variables(16).ValuesAsNumpy()
hourly_soil_moisture_0_to_1cm = hourly.Variables(17).ValuesAsNumpy()
hourly_soil_temperature_54cm = hourly.Variables(18).ValuesAsNumpy()
hourly_soil_moisture_1_to_3cm = hourly.Variables(19).ValuesAsNumpy()
hourly_soil_moisture_3_to_9cm = hourly.Variables(20).ValuesAsNumpy()
hourly_soil_moisture_27_to_81cm = hourly.Variables(21).ValuesAsNumpy()
hourly_soil_moisture_9_to_27cm = hourly.Variables(22).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["rain"] = hourly_rain
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["precipitation_probability"] = hourly_precipitation_probability
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["temperature_180m"] = hourly_temperature_180m
hourly_data["temperature_80m"] = hourly_temperature_80m
hourly_data["temperature_120m"] = hourly_temperature_120m
hourly_data["soil_temperature_0cm"] = hourly_soil_temperature_0cm
hourly_data["soil_temperature_6cm"] = hourly_soil_temperature_6cm
hourly_data["soil_temperature_18cm"] = hourly_soil_temperature_18cm
hourly_data["soil_moisture_0_to_1cm"] = hourly_soil_moisture_0_to_1cm
hourly_data["soil_temperature_54cm"] = hourly_soil_temperature_54cm
hourly_data["soil_moisture_1_to_3cm"] = hourly_soil_moisture_1_to_3cm
hourly_data["soil_moisture_3_to_9cm"] = hourly_soil_moisture_3_to_9cm
hourly_data["soil_moisture_27_to_81cm"] = hourly_soil_moisture_27_to_81cm
hourly_data["soil_moisture_9_to_27cm"] = hourly_soil_moisture_9_to_27cm

hourly_dataframe = pd.DataFrame(data = hourly_data)

# Add datetime components after creating the DataFrame
hourly_dataframe["hour"] = pd.to_datetime(hourly_dataframe["date"]).dt.hour
hourly_dataframe["day"] = pd.to_datetime(hourly_dataframe["date"]).dt.day
hourly_dataframe["month"] = pd.to_datetime(hourly_dataframe["date"]).dt.month
hourly_dataframe["year"] = pd.to_datetime(hourly_dataframe["date"]).dt.year

print("\nHourly data\n", hourly_dataframe)

hourly_dataframe.to_csv('weather_data.csv', index=False)


Coordinates: 52.52000045776367°N 13.419998168945312°E
Elevation: 38.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                           date  temperature_2m  rain  relative_humidity_2m  \
0    2025-11-03 00:00:00+00:00             NaN   NaN                   NaN   
1    2025-11-03 01:00:00+00:00             NaN   NaN                   NaN   
2    2025-11-03 02:00:00+00:00             NaN   NaN                   NaN   
3    2025-11-03 03:00:00+00:00             NaN   NaN                   NaN   
4    2025-11-03 04:00:00+00:00             NaN   NaN                   NaN   
...                        ...             ...   ...                   ...   
2587 2026-02-18 19:00:00+00:00          7.3585   0.0                  83.0   
2588 2026-02-18 20:00:00+00:00          7.3085   0.0                  81.0   
2589 2026-02-18 21:00:00+00:00          7.2085   0.0                  80.0   
2590 2026-02-18 22:00:00+00:00          6.9585   0.0                  81.0   
2591 2026-02-18 23:

In [5]:
df = pd.read_csv('weather_data.csv')
y = df[['rain','date']]
X = df.drop(columns=['rain'])

In [6]:
print("\nFeatures (X)\n", X)
print("\nTarget (y)\n", y)


Features (X)
                            date  temperature_2m  relative_humidity_2m  \
0     2025-11-03 00:00:00+00:00             NaN                   NaN   
1     2025-11-03 01:00:00+00:00             NaN                   NaN   
2     2025-11-03 02:00:00+00:00             NaN                   NaN   
3     2025-11-03 03:00:00+00:00             NaN                   NaN   
4     2025-11-03 04:00:00+00:00             NaN                   NaN   
...                         ...             ...                   ...   
2587  2026-02-18 19:00:00+00:00          7.3585                  83.0   
2588  2026-02-18 20:00:00+00:00          7.3085                  81.0   
2589  2026-02-18 21:00:00+00:00          7.2085                  80.0   
2590  2026-02-18 22:00:00+00:00          6.9585                  81.0   
2591  2026-02-18 23:00:00+00:00          6.6085                  83.0   

      dew_point_2m  apparent_temperature  precipitation  \
0              NaN                   NaN         